# Tugas Praktikum JS14: CNN untuk CIFAR-10 dan MNIST

## Tugas:
1. **Modifikasi model CNN** pada praktikum 2 sehingga didapatkan akurasi testing lebih dari 80%
2. **Buatlah model CNN** untuk klasifikasi dataset MNIST

In [ ]:
# Import libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

print(f"TensorFlow version: {tf.__version__}")
print("Libraries imported successfully!")

---
# TUGAS 1: Modifikasi CNN untuk CIFAR-10 (Akurasi > 80%)

## Strategi untuk meningkatkan akurasi:
1. Menambah layer konvolusi
2. Menggunakan Batch Normalization
3. Menambah Dropout untuk regularisasi
4. Menggunakan Data Augmentation
5. Menggunakan callbacks (EarlyStopping, ReduceLROnPlateau)

In [ ]:
# Load dataset CIFAR-10
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

# Normalize pixel values to be between 0 and 1
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# Class names
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

print("=" * 60)
print("DATASET CIFAR-10")
print("=" * 60)
print(f"Training data shape: {train_images.shape}")
print(f"Testing data shape: {test_images.shape}")
print(f"Number of classes: {len(class_names)}")

# Visualize some samples
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_images[i])
    plt.xlabel(class_names[train_labels[i][0]])
plt.suptitle('Contoh Gambar CIFAR-10', fontsize=14)
plt.tight_layout()
plt.show()

## Model CNN yang Dimodifikasi

Modifikasi dari model praktikum 2:
- Menambah layer konvolusi (dari 3 menjadi 6)
- Menambah Batch Normalization setelah setiap Conv2D
- Menambah Dropout untuk mencegah overfitting
- Menambah neuron di Dense layer

In [ ]:
# Model CNN yang dimodifikasi untuk akurasi > 80%
model_cifar = models.Sequential([
    # Block 1: Conv -> BatchNorm -> Conv -> BatchNorm -> MaxPool -> Dropout
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Block 2: Conv -> BatchNorm -> Conv -> BatchNorm -> MaxPool -> Dropout
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Block 3: Conv -> BatchNorm -> Conv -> BatchNorm -> MaxPool -> Dropout
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

print("=" * 60)
print("ARSITEKTUR MODEL CNN YANG DIMODIFIKASI")
print("=" * 60)
model_cifar.summary()

In [ ]:
# Compile model
model_cifar.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-6,
    verbose=1
)

print("Model compiled successfully!")
print("Callbacks: EarlyStopping, ReduceLROnPlateau")

In [ ]:
# Data Augmentation
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    zoom_range=0.1
)
datagen.fit(train_images)

print("=" * 60)
print("TRAINING MODEL CIFAR-10")
print("=" * 60)
print("Menggunakan Data Augmentation untuk meningkatkan akurasi...")

# Training dengan data augmentation
history_cifar = model_cifar.fit(
    datagen.flow(train_images, train_labels, batch_size=64),
    epochs=50,
    validation_data=(test_images, test_labels),
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

In [ ]:
# Evaluasi model CIFAR-10
test_loss_cifar, test_acc_cifar = model_cifar.evaluate(test_images, test_labels, verbose=0)

print("=" * 60)
print("HASIL EVALUASI MODEL CIFAR-10")
print("=" * 60)
print(f"\n🎯 Test Loss: {test_loss_cifar:.4f}")
print(f"🎯 Test Accuracy: {test_acc_cifar:.4f} ({test_acc_cifar*100:.2f}%)")

if test_acc_cifar > 0.80:
    print(f"\n✅ TARGET TERCAPAI! Akurasi > 80%")
else:
    print(f"\n⚠️ Akurasi belum mencapai 80%, coba tambah epoch atau modifikasi model")

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_cifar.history['accuracy'], label='Training Accuracy')
axes[0].plot(history_cifar.history['val_accuracy'], label='Validation Accuracy')
axes[0].axhline(y=0.8, color='r', linestyle='--', label='Target 80%')
axes[0].set_title('Model CIFAR-10: Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history_cifar.history['loss'], label='Training Loss')
axes[1].plot(history_cifar.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model CIFAR-10: Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---
# TUGAS 2: Model CNN untuk Klasifikasi MNIST

Dataset MNIST berisi gambar digit tulisan tangan (0-9) dengan ukuran 28x28 pixels grayscale.

In [ ]:
# Load dataset MNIST
(mnist_train_images, mnist_train_labels), (mnist_test_images, mnist_test_labels) = datasets.mnist.load_data()

# Reshape untuk CNN (menambah channel dimension)
mnist_train_images = mnist_train_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0
mnist_test_images = mnist_test_images.reshape(-1, 28, 28, 1).astype('float32') / 255.0

print("=" * 60)
print("DATASET MNIST")
print("=" * 60)
print(f"Training data shape: {mnist_train_images.shape}")
print(f"Testing data shape: {mnist_test_images.shape}")
print(f"Number of classes: 10 (digit 0-9)")

# Visualize some samples
plt.figure(figsize=(10, 4))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(mnist_train_images[i].reshape(28, 28), cmap='gray')
    plt.xlabel(f'Label: {mnist_train_labels[i]}')
plt.suptitle('Contoh Gambar MNIST', fontsize=14)
plt.tight_layout()
plt.show()

## Model CNN untuk MNIST

In [ ]:
# Model CNN untuk MNIST
model_mnist = models.Sequential([
    # Block 1: Conv -> BatchNorm -> MaxPool -> Dropout
    layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(28, 28, 1)),
    layers.BatchNormalization(),
    layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Block 2: Conv -> BatchNorm -> MaxPool -> Dropout
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.25),
    
    # Fully Connected Layers
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

print("=" * 60)
print("ARSITEKTUR MODEL CNN UNTUK MNIST")
print("=" * 60)
model_mnist.summary()

In [ ]:
# Compile model MNIST
model_mnist.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Callbacks
early_stop_mnist = EarlyStopping(
    monitor='val_accuracy',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

reduce_lr_mnist = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=3,
    min_lr=1e-6,
    verbose=1
)

print("=" * 60)
print("TRAINING MODEL MNIST")
print("=" * 60)

# Training
history_mnist = model_mnist.fit(
    mnist_train_images, mnist_train_labels,
    epochs=20,
    batch_size=128,
    validation_split=0.1,
    callbacks=[early_stop_mnist, reduce_lr_mnist],
    verbose=1
)

In [ ]:
# Evaluasi model MNIST
test_loss_mnist, test_acc_mnist = model_mnist.evaluate(mnist_test_images, mnist_test_labels, verbose=0)

print("=" * 60)
print("HASIL EVALUASI MODEL MNIST")
print("=" * 60)
print(f"\n🎯 Test Loss: {test_loss_mnist:.4f}")
print(f"🎯 Test Accuracy: {test_acc_mnist:.4f} ({test_acc_mnist*100:.2f}%)")

# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Accuracy
axes[0].plot(history_mnist.history['accuracy'], label='Training Accuracy')
axes[0].plot(history_mnist.history['val_accuracy'], label='Validation Accuracy')
axes[0].set_title('Model MNIST: Accuracy')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Accuracy')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Loss
axes[1].plot(history_mnist.history['loss'], label='Training Loss')
axes[1].plot(history_mnist.history['val_loss'], label='Validation Loss')
axes[1].set_title('Model MNIST: Loss')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Prediksi beberapa sampel MNIST
predictions = model_mnist.predict(mnist_test_images[:10])
predicted_labels = np.argmax(predictions, axis=1)

# Visualisasi prediksi
plt.figure(figsize=(12, 5))
for i in range(10):
    plt.subplot(2, 5, i+1)
    plt.imshow(mnist_test_images[i].reshape(28, 28), cmap='gray')
    color = 'green' if predicted_labels[i] == mnist_test_labels[i] else 'red'
    plt.title(f'Pred: {predicted_labels[i]}\nTrue: {mnist_test_labels[i]}', color=color)
    plt.axis('off')
plt.suptitle('Hasil Prediksi MNIST (Hijau=Benar, Merah=Salah)', fontsize=14)
plt.tight_layout()
plt.show()

---
## Ringkasan Hasil

In [ ]:
# ============================================================
# RINGKASAN HASIL TUGAS PRAKTIKUM
# ============================================================

print("=" * 70)
print("📊 RINGKASAN HASIL TUGAS PRAKTIKUM")
print("=" * 70)

print(f"""
╔══════════════════════════════════════════════════════════════════════╗
║                    TUGAS 1: CIFAR-10 (Akurasi > 80%)                 ║
╠══════════════════════════════════════════════════════════════════════╣
║  Test Accuracy: {test_acc_cifar*100:.2f}%                                            ║
║  Test Loss: {test_loss_cifar:.4f}                                                ║
║  Status: {'✅ TARGET TERCAPAI!' if test_acc_cifar > 0.80 else '⚠️ Belum tercapai'}                                          ║
╠══════════════════════════════════════════════════════════════════════╣
║  Modifikasi yang dilakukan:                                          ║
║  - 6 layer konvolusi (dari 3)                                        ║
║  - Batch Normalization setelah setiap Conv2D                         ║
║  - Dropout (0.25 setelah pooling, 0.5 di FC)                         ║
║  - Data Augmentation (rotation, shift, flip, zoom)                   ║
║  - Callbacks (EarlyStopping, ReduceLROnPlateau)                      ║
╚══════════════════════════════════════════════════════════════════════╝

╔══════════════════════════════════════════════════════════════════════╗
║                    TUGAS 2: MNIST Classification                     ║
╠══════════════════════════════════════════════════════════════════════╣
║  Test Accuracy: {test_acc_mnist*100:.2f}%                                            ║
║  Test Loss: {test_loss_mnist:.4f}                                                ║
╠══════════════════════════════════════════════════════════════════════╣
║  Arsitektur:                                                         ║
║  - 4 layer konvolusi (32-32-64-64 filters)                           ║
║  - Batch Normalization                                               ║
║  - MaxPooling dan Dropout                                            ║
║  - Dense layers (256-128-10)                                         ║
╚══════════════════════════════════════════════════════════════════════╝
""")